In [ ]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import datetime

def categorize(x):
    if x <= 30: return 0
    elif x <= 50: return 1
    else: return 2

def preprocess_3h(df):

    x = df.copy()

    x['category'] = x['Kp*10'].apply(categorize)

    x['dttm'] = x.apply(
        lambda y: datetime.datetime(
            int(y.year), int(y.month), 
            int(y.day), int(y['hour from']), 0), axis=1)
    
    x_3h = x.sort_values(by='dttm').iloc[:: 3]
    return x_3h

ROOT_PATH = '/content/drive/MyDrive/kp_data'

import os
import glob

files = glob.glob(os.path.join(ROOT_PATH, '*'))

files = dict(map(lambda x: (x.split('/')[-1], x), files))

df = pd.read_csv(
    files['All_browse_data_без_погружения_19971021_20211231_с_пропусками.csv'], 
    encoding='cp1251', na_values='N').pipe(preprocess_3h)
print(df.shape)
df.head()

(70776, 65)


,year,month,day,hour from,hour to,doySin,hourSin,doyCos,hourCos,Dst,...,lg(E>2 MeV),E>2 MeV G16,lg(E>2 MeV) G16,CH_rca (193),CH_rca (211),SW_spd_frcst (193),SW_spd_frcst (211),Unnamed: 62,category,dttm
0,1997,10,21,0,1,-0.505271,0.991445,-0.862961,-0.130526,-14,...,0.715307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1997-10-21 00:00:00
3,1997,10,21,3,4,-0.507126,0.608761,-0.861872,-0.793353,-13,...,0.318573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1997-10-21 03:00:00
6,1997,10,21,6,7,-0.508980,-0.130526,-0.860779,-0.991445,-15,...,0.070838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1997-10-21 06:00:00
9,1997,10,21,9,10,-0.510831,-0.793353,-0.859681,-0.608761,-13,...,0.501584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1997-10-21 09:00:00
12,1997,10,21,12,13,-0.512679,-0.991445,-0.858580,0.130526,-10,...,1.011042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1997-10-21 12:00:00


In [ ]:
all_variables = [
    'Kp*10', 'Dst', 'B_x', 'B_gsm_y', 'B_gsm_z', 
    'B_magn', 'SW_spd', 'H_den_SWP', 'dttm', 'category'
 ]

predictors  = [
    'Kp*10', 'Dst', 'B_x', 'B_gsm_y', 'B_gsm_z', 
    'B_magn', 'SW_spd', 'H_den_SWP',
 ]

def simple_preprocessing(df):
    x = df.copy()

    x = x.bfill()

    return x

def _choose_suffix_name(forward: bool, suffix_name: str) -> str:
    if suffix_name is not None:
        return suffix_name
    else:
        return 'lead' if forward else 'lag'

def _trim(df: pd.DataFrame, forward: bool, trim: bool, lags: int) -> pd.DataFrame:
    if trim and forward:
        return df.iloc[: -lags]
    elif trim:
        return df.iloc[lags: ]
    else:
        return df

def add_lags(df: pd.DataFrame, subset: list=None, forward: bool=False,
             lags: int=1, trim: bool=False, suffix_name: str=None,
             return_cols=False) -> pd.DataFrame:

    suffix_name = _choose_suffix_name(forward, suffix_name)

    x = df.copy()

    digits = len(str(lags))

    columns = []

    if not isinstance(lags, int):
        raise ValueError(f'Lags should be int, {type(lags)} type prodided')
    elif lags < 0:
        raise ValueError(f'Lags should be non-negative')
    elif lags == 0:
        return x
    elif subset is None:
        for i in range(1, lags + 1):
            lag = - i if forward else i
            index = str(i).zfill(digits)
            column_suffix = f'_{suffix_name}_{index}'

            x = x.join(x.shift(lag).add_suffix(column_suffix))
        
        columns = x.columns.tolist()

    elif isinstance(subset, list):
        for i in range(1, lags + 1):
            lag = - i if forward else i
            index = str(i).zfill(digits)
            column_suffix = f'_{suffix_name}_{index}'
            tmp = x.loc[:, subset].shift(lag).add_suffix(column_suffix)
            columns.extend(tmp.columns)
            x = x.join(tmp)

    elif isinstance(subset, str):
        for i in range(1, lags + 1):
            lag = - i if forward else i
            index = str(i).zfill(digits)
            column_name = f'{subset}_{suffix_name}_{index}'
            columns.append(column_name)

            x = x.join(x.loc[:, subset].shift(lag).rename(column_name))
    else:
        raise ValueError(f'Subset should be str or list, providided type {type(subset)}')

    if return_cols:
        return _trim(x, forward, trim, lags), columns
    else:
        return _trim(x, forward, trim, lags)


def get_train_test(df: pd.DataFrame, columns: list, 
                   forward_steps: int, backward_steps: int, 
                   last: str='24m') -> tuple:

    ts_df_back, lag_cols = add_lags(df, lags=backward_steps, forward=False, 
                                    trim=True, subset=columns, 
                                    return_cols=True)
    lag_cols.extend(columns)
    ts_df_back_test = ts_df_back.set_index('dttm').last(last)

    index_test = ts_df_back_test.index
    ts_df_back_train = ts_df_back.set_index('dttm').drop(index_test)

    df_train, lead_cols = add_lags(ts_df_back_train, lags=forward_steps,
                                   forward=True, trim=True, 
                                   subset='category', return_cols=True)
    df_test, lead_cols = add_lags(ts_df_back_test, lags=forward_steps, 
                                  forward=True, trim=True, 
                                  subset='category', return_cols=True)

    return df_train.reset_index(), lag_cols, df_test.reset_index(), lead_cols

from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
from sklearn.metrics import f1_score

from types import FunctionType

def columnwise_score(scoring_func: FunctionType, 
                     preds_df: pd.DataFrame, 
                     true_df: pd.DataFrame,
                     **kwargs) -> pd.Series:

    score = pd.Series(dtype='float64')
    for (column_pred, y_pred), (columns_true, y_true) in zip(preds_df.iteritems(), 
                                                             true_df.iteritems()):
        score[column_pred] = scoring_func(y_pred, y_true, **kwargs)
    
    return score

In [ ]:
df_train, lag_cols, df_test, lead_cols = get_train_test(
    df.pipe(simple_preprocessing), predictors, 24 // 3, 24)

In [ ]:
X_train, y_train = df_train[lag_cols], df_train[lead_cols]
X_test, y_test = df_test[lag_cols], df_test[lead_cols]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers as L
from tensorflow.keras import optimizers as opt
from tensorflow.keras import callbacks as callbacks

from tensorflow.random import set_seed

In [ ]:
def get_model(input_shape: tuple, units_array: list=[10], n_classes: int=3,
              optimizer: str='adam') -> Sequential:

    model = Sequential([
        L.Input(shape=input_shape),
        *(L.Dense(units=units, activation='relu') for units in units_array),
        L.Dense(units=3, activation='sigmoid')  
    ])

    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

In [ ]:
get_model((2, ), [10, 10])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
shape = (X_train_scaled.shape[1], )

preds_df = {}

for col in lead_cols:
    y_dummy = pd.get_dummies(y_train[col])
    set_seed(17)
    model = get_model(shape, units_array=[128,])

    callbacks_list = [
        callbacks.EarlyStopping(monitor='val_loss', 
                                restore_best_weights=True, patience=10),
    ]

    model.fit(X_train_scaled, y_dummy, epochs=60, 
              callbacks=callbacks_list, validation_split=.1, )
    
    preds = model.predict(X_test_scaled).argmax(axis=1)

    preds_df[col] = preds

Epoch 1/60
1831/1831 [==============================] - 5s 2ms/step - loss: 0.3339 - val_loss: 0.1642
Epoch 2/60
1831/1831 [==============================] - 4s 2ms/step - loss: 0.2968 - val_loss: 0.1603
Epoch 3/60
1831/1831 [==============================] - 4s 2ms/step - loss: 0.2864 - val_loss: 0.1609
Epoch 4/60
1831/1831 [==============================] - 10s 5ms/step - loss: 0.2793 - val_loss: 0.1580
Epoch 5/60
1831/1831 [==============================] - 7s 4ms/step - loss: 0.2719 - val_loss: 0.1616
Epoch 6/60
1831/1831 [==============================] - 4s 2ms/step - loss: 0.2659 - val_loss: 0.1650
Epoch 7/60
1831/1831 [==============================] - 5s 2ms/step - loss: 0.2598 - val_loss: 0.1628
Epoch 8/60
1831/1831 [==============================] - 5s 3ms/step - loss: 0.2523 - val_loss: 0.1637
Epoch 9/60
1831/1831 [==============================] - 5s 3ms/step - loss: 0.2454 - val_loss: 0.1717
Epoch 10/60
1831/1831 [==============================] - 5s 3ms/step - loss: 0.23

In [ ]:
df_train, lag_cols, df_test, lead_cols = get_train_test(
    df.pipe(simple_preprocessing), predictors, 24 // 3, 24)



In [ ]:
f1_weighted_res = columnwise_score(f1_score, preds_df, y_test, average='macro')

In [ ]:
f1_weighted_res.to_csv(os.path.join(ROOT_PATH, 'mlp.csv'))

In [ ]:
f1_weighted_res

category_lead_1    0.617563
category_lead_2    0.482916
category_lead_3    0.359063
category_lead_4    0.330083
category_lead_5    0.369420
category_lead_6    0.323093
category_lead_7    0.323651
category_lead_8    0.326930
dtype: float64